In [ ]:
!pip install pymupdf

In [ ]:
!pip install langchain llama_index pinecone-client faiss-cpu #transformers

In [ ]:
!pip install langchain-community

In [ ]:
!pip install sentence_transformers

In [ ]:
import fitz  # PyMuPDF
import re

# Define the PDF file path
pdf_path = '/content/History_Ancient_Medieval_Nepal.pdf'

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text")
    return text

def clean_text(text):
    # Remove newlines and unnecessary whitespace
    text = re.sub(r'\n+', ' ', text)
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters
    text = re.sub(r'[^\w\s,.]', '', text)
    return text.strip()

def split_text(text, chunk_size=1000, overlap=200):
    # Split text into chunks of specified size with overlap
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

# Call the function to extract text from the PDF
text = extract_text_from_pdf(pdf_path)

# Clean the extracted text
cleaned_text = clean_text(text)

# Split the cleaned text into chunks
chunks = split_text(cleaned_text, chunk_size=1000, overlap=200)

# Print the first few chunks to verify
for i, chunk in enumerate(chunks[:3]):  # Print the first 3 chunks for verification
    print(f"Chunk {i+1}:\n{chunk}\n")




Chunk 1:
THE HISTORY OF ANCIENT AND MEDIEVAL NEPAL In Nutshell with Some Comparative Traces ol Foreign History 1972 BOOK 1 D. B. SHRESTHA  C B. SINGH The History of Ancient and Medieval NEPAL In n Nutshell with Some Comparative Traces of Foreign History m D.B. SHRESTHA  C.B. SINGH Published by the Authors ALL RIGHTS RESERVED First Edition 1000 Copies 1972 A. D. 45 Printed at HMG Press, Kathmandi a ef 3 F2 u n aa 13a 57 ft a I The Position of Nepal in Asia His Majesty King Birendra Bir Bikram Shah Dev of Nepal Preface This book History of Ancient and Medieval Nepal does not claim to be a revealing book which seeks to throw light on the sofar unrevealed historical facts or on the controversial topics. It has been written on the basis of facts and figures so far published or handed down to us through the untiring efforts of the research scholars who have devoted their time and energy to the factfinding activities which consist in excavations or the study of ancient stone tablets and other

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

def create_faiss_index(docs):
    """Creates and saves a FAISS index from document chunks."""
    # Initialize HuggingFace embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Create a list of Document objects
    documents = [Document(page_content=doc) for doc in docs]

    # Create the FAISS vector store
    vectorstore = FAISS.from_documents(documents, embeddings)

    # Save the FAISS index for future use
    vectorstore.save_local("faiss_index")
    print("FAISS index created and saved successfully!")

# Example usage
create_faiss_index(chunks)

FAISS index created and saved successfully!


In [ ]:
!pip install --upgrade langchain
%pip install -qU langchain-google-genai

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import warnings
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Suppress the FutureWarning related to tokenization
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")

def create_faiss_index(docs):
    """Creates and saves a FAISS index from document chunks."""
    # Initialize HuggingFace embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Create a list of Document objects
    documents = [Document(page_content=doc) for doc in docs]

    # Create the FAISS vector store
    vectorstore = FAISS.from_documents(documents, embeddings)

    # Save the FAISS index for future use
    vectorstore.save_local("faiss_index")
    print("FAISS index created and saved successfully!")

def load_vectorstore():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    return vectorstore

def chatbot():
    vectorstore = load_vectorstore()

    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        api_key=''
    )

    # Define the template for chatbot responses
    template = """
    You are a virtual AI assistant. Based on the following context, answer the user's query directly and accurately.
    Give me a paragraph answer.
    Context: {context}

    User query: {input}

    Answer:
    """

    print("Chatbot is ready! Type 'stop' or 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['stop', 'exit']:
            break

        # Search for relevant documents from the FAISS index
        docs = vectorstore.similarity_search(user_input, k=2)  # Retrieve top 2 similar documents

        # Print the chunks used for the answer
        for i, doc in enumerate(docs):
            print(f"Chunk {i + 1}: {doc.page_content[:100]}...")  # Print a snippet of the chunk (first 100 chars)

        # Concatenate the content of the top documents (limiting to 1000 chars per doc)
        context = "\n".join([doc.page_content[:1000] for doc in docs])

        # Create a prompt using the context and user input
        prompt_template = PromptTemplate.from_template(template=template)
        formatted_prompt = prompt_template.format(context=context, input=user_input)

        # Generate the response using the LLM
        answer = llm.invoke(formatted_prompt)
        print(f"Bot: {answer}")

# Start the chatbot
chatbot()


Chatbot is ready! Type 'stop' or 'exit' to end the conversation.


Bot: content='    This text does not mention anyone named Pascal Rai. This text describes the lives and accomplishments of various kings. \n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-ace51ba9-7500-45c5-9553-5b6f3f949e76-0' usage_metadata={'input_tokens': 501, 'output_tokens': 21, 'total_tokens': 522}
Bot: content="The Lichchhavi period is considered a golden age in Nepal's history due to its significant advancements in various fields. This era saw a flourishing of art and architecture, evidenced by magnificent